For testing patent retrieval based on claim

importing libraries

In [6]:
import faiss
import pandas as pd
from sentence_transformers import SentenceTransformer
import gc
import os
from dotenv import load_dotenv

Importing env

In [7]:
load_dotenv()

cosine_index = os.getenv("INDEX_PATH_COSINE")
l2_index = os.getenv("INDEX_PATH_L2")
metadata_path = os.getenv("METADATA_PATH")

Loading index, metadata and embedding model

<p>Two FAISS indices were created, one using cosine similarity, and one using Euclidean distance.</p>

In [ ]:
index = faiss.read_index(cosine_index)
#index = faiss.read_index(l2_index)
metadata = pd.read_csv(metadata_path, low_memory=False)
model = SentenceTransformer("..\\local_minilm_l6_v2")

We are importing table:
check.csv - A table that is created in helpers/create-check-table.py
The same table is after finishing this code again updated with innography information using script helpers/table-to-check.py

In this part we are retrieving top 10 retrieved patent_ids based on the claim. The ten patents are unique patents (we ignore the repeated patents and therefore we are retrieving 50 ids, then picking top 10 based on the distance), where the first one is supposed to be a patent whose claim is used as a query.

Then this table is used in helpers/table-to-check.py to see how many of 10 retrieved patents are found in 100 patents that are retrieved from Innography.


In [12]:
df = pd.read_csv("..\\result tables\\check1.csv")

k = 10

retrieved_ids = []
found_itself_flags = []

for i, row in df.iterrows():
    claim = row['claim']
    true_patent_id = str(row['patent_id']).strip()

    if pd.isna(claim) or not claim.strip():
        retrieved_ids.append("")
        found_itself_flags.append("N")
        continue

    search_k = k * 5 
    query_embedding = model.encode([claim], normalize_embeddings=True)
    distances, indices = index.search(query_embedding.astype("float32"), search_k)
    
    top_k_metadata = metadata.iloc[indices[0]]
    all_ids = top_k_metadata['patent_id'].astype(str)

    unique_ids = list(dict.fromkeys(all_ids))[:k]
    
    retrieved_ids.append(" | ".join(unique_ids))
    found_itself_flags.append("Y" if true_patent_id in unique_ids else "N")


df['top 10 retrieved patents'] = retrieved_ids
df['found itself (Y/N)'] = found_itself_flags

df.to_csv("..\\result tables\\check2_with_retrieved_cos.csv", index=False)

print("Retrieval done.")

Retrieval done.


Due to memory issues, to change index (cosine or distance), it is required to first delete the loaded index, and then reload again the desired index.

In [ ]:

try:
    del index
except NameError:
    pass

gc.collect()

index = faiss.read_index(l2_index)